<a href="https://colab.research.google.com/github/dilanbakr/machineTranslate/blob/main/Makine_%C3%87evirisi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==1.4.0

     |████████████████████████████████| 41.2MB 98kB/s 
     |████████████████████████████████| 1.7MB 48.5MB/s 
     |████████████████████████████████| 890kB 40.6MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107220 sha256=28de974c7e7969ff3d461df4f3b73e1f4f6876b8690796930639eca019c8b9c7
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.2.0
    Uninstalling bleach-3.2.0:
      Successfully uninstalled bleach-3.2.0
  Found existing installation: tensorflow 2.0.0
    Uninstalling tensorflow-2.0.0:
      Successfully uninstalled tensorflow-2.0.0


In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [ ]:
mark_start = 'ssss '
mark_end = ' eeee'

In [ ]:
data_src = []
data_dest = []

In [ ]:
for  line in open("tur.txt", encoding = "utf-8"):
  en_text, tr_text = line.rstrip().split('\t')
  tr_text = mark_start + tr_text + mark_end
  data_src.append(en_text)
  data_dest.append(tr_text)


In [ ]:
data_src[70]

'Get Tom.'

In [ ]:
data_dest[70]

"ssss Tom'u alın. eeee"

In [ ]:
len(data_src)

473035

In [ ]:
len(data_dest)

473035

In [ ]:
class TokenizerWrap(Tokenizer):
    def __init__(self, texts, padding, reverse=False, num_words=None):
        Tokenizer.__init__(self, num_words=num_words)
        
        self.fit_on_texts(texts)
        
        self.index_to_word = dict(zip(self.word_index.values(), self.word_index.keys()))
        
        self.tokens = self.texts_to_sequences(texts)
        
        if reverse:
            self.tokens = [list(reversed(x)) for x in self.tokens]
            truncating = 'pre'
        else:
            truncating = 'post'
            
        self.num_tokens = [len(x) for x in self.tokens]
        self.max_tokens = np.mean(self.num_tokens) + 2 * np.std(self.num_tokens)
        self.max_tokens = int(self.max_tokens)
        
        self.tokens_padded = pad_sequences(self.tokens,
                                           maxlen=self.max_tokens,
                                           padding=padding,
                                           truncating=truncating)
        
    def token_to_word(self, token):
        word = ' ' if token == 0 else self.index_to_word[token]
        return word
    
    def tokens_to_string(self, tokens):
        words = [self.index_to_word[token] for token in tokens if token != 0]
        text = ' '.join(words)
        return text
    
    def text_to_tokens(self, text, padding, reverse=False):
        tokens = self.texts_to_sequences([text])
        tokens = np.array(tokens)
        
        if reverse:
            tokens = np.flip(tokens, axis=1)
            truncating = 'pre'
        else:
            truncating = 'post'
            
        tokens = pad_sequences(tokens,
                               maxlen=self.max_tokens,
                               padding=padding,
                               truncating=truncating)
        
        return tokens


In [ ]:
data_src[100]

'I drove.'

In [ ]:
tokenizer_src = TokenizerWrap(texts=data_src,
                              padding='pre',
                              reverse=True,
                              num_words=None)

In [ ]:
tokenizer_dest = TokenizerWrap(texts=data_dest,
                              padding='post',
                              reverse=False,
                              num_words=None)

In [ ]:
tokens_src = tokenizer_src.tokens_padded
tokens_dest = tokenizer_dest.tokens_padded
print(tokens_src.shape)
print(tokens_dest.shape)

(473035, 11)
(473035, 10)


In [ ]:
tokens_dest[200000]

array([   1, 2391,    4,   18, 4127,   48,    2,    0,    0,    0],
      dtype=int32)

In [ ]:
tokens_src[200000]

array([   0,    0,    0,    0,    0,    0, 1028,  113,   95,    5,   39],
      dtype=int32)

In [ ]:
tokenizer_src.tokens_to_string(tokens_src[200000])

'missing anything see you can'

In [ ]:
tokenizer_dest.tokens_to_string(tokens_dest[200000])

'ssss eksik bir şey görebiliyor musun eeee'

In [ ]:
token_start = tokenizer_dest.word_index[mark_start.strip()]
token_start

1

In [ ]:
token_end = tokenizer_dest.word_index[mark_end.strip()]
token_end

2

In [ ]:
encoder_input_data = tokens_src

In [ ]:
decoder_input_data = tokens_dest[:, :-1]
decoder_output_data = tokens_dest[:, 1:]

In [ ]:
encoder_input_data[200000]

array([   0,    0,    0,    0,    0,    0, 1028,  113,   95,    5,   39],
      dtype=int32)

In [ ]:
decoder_input_data[200000]

array([   1, 2391,    4,   18, 4127,   48,    2,    0,    0], dtype=int32)

In [ ]:
decoder_output_data[200000]

array([2391,    4,   18, 4127,   48,    2,    0,    0,    0], dtype=int32)

In [ ]:
tokenizer_dest.tokens_to_string(decoder_input_data[200000])

'ssss eksik bir şey görebiliyor musun eeee'

In [ ]:
num_encoder_words = len(tokenizer_src.word_index)
num_decoder_words = len(tokenizer_dest.word_index)

In [ ]:
num_encoder_words

21315

In [ ]:
num_decoder_words

94058

In [ ]:
embedding_size = 100

In [ ]:
word2vec = {}
with open('glove.6B.100d.txt', encoding='UTF-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec

In [ ]:
embedding_matrix = np.random.uniform(-1, 1, (num_encoder_words, embedding_size))
for word, i in tokenizer_src.word_index.items():
    if i < num_encoder_words:
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

(21315, 100)

In [ ]:
encoder_input = Input(shape=(None,), name='encoder_input')

In [ ]:
encoder_embedding = Embedding(input_dim=num_encoder_words,
                              output_dim=embedding_size,
                              weights=[embedding_matrix],
                              trainable=True,
                              name='encoder_embedding') #bu kısmı değiştir sonra embedding matrix olmadan yap bi!

In [ ]:
state_size = 256

In [ ]:
encoder_gru1 = GRU(state_size, name='encoder_gru1', return_sequences=True)
encoder_gru2 = GRU(state_size, name='encoder_gru2', return_sequences=True)
encoder_gru3 = GRU(state_size, name='encoder_gru3', return_sequences=False)

In [ ]:
#düşünce vectörü alıyoruz yani throuht vector
def connect_encoder():
    net = encoder_input
    
    net = encoder_embedding(net)
    
    net = encoder_gru1(net)
    net = encoder_gru2(net)
    net = encoder_gru3(net)
    
    encoder_output = net
    
    return encoder_output

In [ ]:
encoder_output = connect_encoder()

In [ ]:
decoder_initial_state = Input(shape=(state_size,), name='decoder_initial_state')

In [ ]:
decoder_input = Input(shape=(None,), name='decoder_input')

In [ ]:
decoder_embedding = Embedding(input_dim=num_decoder_words,
                              output_dim=embedding_size,
                              name='decoder_embedding')

In [ ]:
decoder_gru1 = GRU(state_size, name='decoder_gru1', return_sequences=True)
decoder_gru2 = GRU(state_size, name='decoder_gru2', return_sequences=True)
decoder_gru3 = GRU(state_size, name='decoder_gru3', return_sequences=True)

In [ ]:
decoder_dense = Dense(num_decoder_words,
                      activation='linear',
                      name='decoder_output')

In [ ]:
def connect_decoder(initial_state):
    net = decoder_input
    
    net = decoder_embedding(net)
    
    net = decoder_gru1(net, initial_state=initial_state)
    net = decoder_gru2(net, initial_state=initial_state)
    net = decoder_gru3(net, initial_state=initial_state)
    
    decoder_output = decoder_dense(net)
    
    return decoder_output

In [ ]:
decoder_output = connect_decoder(initial_state=encoder_output)

model_train = Model(inputs=[encoder_input, decoder_input], outputs=[decoder_output])

In [ ]:
model_encoder = Model(inputs=[encoder_input], outputs=[encoder_output])

In [ ]:
decoder_output = connect_decoder(initial_state=decoder_initial_state)

model_decoder = Model(inputs=[decoder_input, decoder_initial_state], outputs=[decoder_output])

In [ ]:
def sparse_cross_entropy(y_true, y_pred):
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
    loss_mean = tf.reduce_mean(loss)
    return loss_mean

In [ ]:
optimizer = RMSprop(lr=1e-3)

In [ ]:
decoder_target = tf.placeholder(dtype='int32', shape=(None,None))

In [ ]:
model_train.compile(optimizer=optimizer,
                    loss=sparse_cross_entropy,
                    target_tensors=[decoder_target])

In [ ]:
path_checkpoint = 'checkpoint.keras'
checkpoint = ModelCheckpoint(filepath=path_checkpoint, save_weights_only=True)

In [ ]:
try:
    model_train.load_weights(path_checkpoint)
except Exception as error:
    print('Checkpoint yüklenirken hata oluştu. Eğitime sıfırdan başlanıyor.')
    print(error)

Checkpoint yüklenirken hata oluştu. Eğitime sıfırdan başlanıyor.
Unable to open file (unable to open file: name = 'checkpoint.keras', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)


In [ ]:
encoder_input_data

array([[   0,    0,    0, ...,    0,    0, 2736],
       [   0,    0,    0, ...,    0,    0, 2736],
       [   0,    0,    0, ...,    0,    0,  480],
       ...,
       [   4, 7097,   49, ..., 1641,  461, 1608],
       [ 178,    5, 1323, ...,   31,    5,   72],
       [ 682,    5,    8, ...,   81,  304,   72]], dtype=int32)

In [ ]:
x_data = {'encoder_input': encoder_input_data, 'decoder_input': decoder_input_data}

In [ ]:
y_data = {'decoder_output': decoder_output_data}

In [ ]:
model_train.fit(x=x_data,
                y=y_data,
                batch_size=256,
                epochs=0,
                callbacks=[checkpoint])

In [ ]:
def translate(input_text, true_output_text=None):
    input_tokens = tokenizer_src.text_to_tokens(text=input_text,
                                                reverse=True,
                                                padding='pre')
    
    initial_state = model_encoder.predict(input_tokens)
    
    max_tokens = tokenizer_dest.max_tokens
    
    decoder_input_data = np.zeros(shape=(1, max_tokens), dtype=np.int)
    
    token_int = token_start
    output_text = ''
    count_tokens = 0
    
    while token_int != token_end and count_tokens < max_tokens:
        decoder_input_data[0, count_tokens] = token_int
        x_data = {'decoder_initial_state': initial_state, 'decoder_input': decoder_input_data}
        
        decoder_output = model_decoder.predict(x_data)
        
        token_onehot = decoder_output[0, count_tokens, :]
        token_int = np.argmax(token_onehot)
        
        sampled_word = tokenizer_dest.token_to_word(token_int)
        output_text += ' ' + sampled_word
        count_tokens += 1
        
    print('Input text:')
    print(input_text)
    print()
    
    print('Translated text:')
    print(output_text)
    print()
    
    if true_output_text is not None:
        print('True output text:')
        print(true_output_text)
        print()

In [ ]:
translate(input_text=data_src[400000], true_output_text=data_dest[400000])

Input text:
You are telling it second hand, aren't you?

Translated text:
 sandalyemi zorlanmadı zorlanmadı nezaketsizdi kazanabilip kazanabilip kazanabilip kazanabilip kazanabilip kazanabilip

True output text:
ssss Onu dolaylı olarak anlatıyorsun, değil mi? eeee



In [ ]:
translate(input_text='What is your name?')

Input text:
What is your name?

Translated text:
 şelaleye zorlanmadı zorlanmadı kazanabilip kazanabilip kazanabilip kazanabilip kazanabilip kazanabilip kazanabilip

